In [1]:
using Pkg
Pkg.activate(pwd())

  Activating project at `c:\Users\pecci\Code\MACRO-dev\prototypes\PrincetonUniversity_Prototypes`

In [2]:
@enum Commodity Power Hydrogen

In [3]:
instances(Commodity)

(Power, Hydrogen)

In [4]:
using JuMP

In [5]:
abstract type AbstractResource end

In [6]:
const time_resolution_map = Dict(Power=>1,Hydrogen=>4);
const TimeLength = 8760;

In [7]:
function time_resolution(T::Commodity)   
    return Int(TimeLength/time_resolution_map[T])
end

time_resolution (generic function with 1 method)

In [8]:
resource_type(g::AbstractResource) = g.Commodity;

In [9]:
Base.@kwdef mutable struct Resource <: AbstractResource
    Commodity::Commodity
    Node::Int64;
    R_ID::Int64;
    min_capacity::Float64 = 0.0
    max_capacity::Float64 = 100.0
    existing_capacity::Float64 = 0.0
    PolicyMembership::Vector{String} = [""]
    CanExpand::Bool = true
    CanRetire::Bool = true
    InvCost::Float64 = 0.0;
    FOMCost::Float64 = 0.0;
    VOMCost::Float64 = 0.0;
    _new_capacity::Array{VariableRef}  = VariableRef[]
    _ret_capacity::Array{VariableRef} = VariableRef[]
    _injection::Array{VariableRef} = Array{VariableRef}(undef,time_resolution(Commodity))
end

Resource

In [10]:
#### TODO : VariableResource struct should "inherit" fileds from Resource using macro forward. 
Base.@kwdef mutable struct VariableResource <: AbstractResource
    Commodity::Commodity
    Node::Int64;
    R_ID::Int64;
    min_capacity::Float64 = 0.0
    max_capacity::Float64 = 100.0
    existing_capacity::Float64 = 0.0
    PolicyMembership::Vector{String} = [""]
    CanExpand::Bool = true
    CanRetire::Bool = true
    InvCost::Float64 = 0.0;
    FOMCost::Float64 = 0.0;
    VOMCost::Float64 = 0.0;
    capacity_factor :: Vector{Float64} = ones(time_resolution(Commodity))
    _new_capacity::Array{VariableRef}  = VariableRef[]
    _ret_capacity::Array{VariableRef} = VariableRef[]
    _injection::Array{VariableRef} = Array{VariableRef}(undef,time_resolution(Commodity))
end

VariableResource

In [12]:
new_capacity(g::AbstractResource) = g._new_capacity[1];
function new_capacity!(g::AbstractResource,v::VariableRef)
    g._new_capacity= [v];
    return nothing
end

new_capacity! (generic function with 1 method)

In [13]:
ret_capacity(g::AbstractResource) = g._ret_capacity[1];
function ret_capacity!(g::AbstractResource,v::VariableRef)
    g._ret_capacity = [v];
    return nothing
end


ret_capacity! (generic function with 1 method)

In [14]:
injection(g::AbstractResource) = g._injection;
function injection!(g::AbstractResource,v::Array{VariableRef})
    g._injection = v;
    return nothing
end

injection! (generic function with 1 method)

In [15]:
function add_capacity_variables!(g::AbstractResource,model::Model)
    new_capacity!(g, @variable(model,lower_bound=0.0,base_name="vCAP_$(resource_type(g))_$(g.R_ID)"))
    ret_capacity!(g, @variable(model,lower_bound=0.0,base_name="vRETCAP_$(resource_type(g))_$(g.R_ID)"))
end


add_capacity_variables! (generic function with 1 method)

In [16]:
# function time_resolution(c::Commodity)
#     isa(c,Power) && return 24


In [17]:
solar = VariableResource(Commodity=Power,Node = 1,R_ID=1,InvCost=85300.0,FOMCost =18760)

VariableResource(Power, 1, 1, 0.0, 100.0, 0.0, [""], true, true, 85300.0, 18760.0, 0.0, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], VariableRef[], VariableRef[], VariableRef[#undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef  …  #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef, #undef])

In [18]:
solar.capacity_factor

8760-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [19]:
model = Model()

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [20]:
add_capacity_variables!(solar,model)

In [21]:
println(model)

Feasibility
Subject to


 vCAP_Power_1 >= 0
 vRETCAP_Power_1 >= 0

